<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/feature%2Fadd-torch-multi-task-learning/multi_task_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm run_ner.py tasks.py utils_ner.py
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-torch-multi-task-learning/run_ner.py
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-torch-multi-task-learning/tasks.py
!wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/feature/add-torch-multi-task-learning/utils_ner.py

rm: cannot remove 'run_ner.py': No such file or directory
rm: cannot remove 'tasks.py': No such file or directory
rm: cannot remove 'utils_ner.py': No such file or directory
run_ner.py          100%[===================>]  15.71K  --.-KB/s    in 0.001s  
tasks.py            100%[===================>]   5.38K  --.-KB/s    in 0s      
utils_ner.py        100%[===================>]  22.26K  --.-KB/s    in 0.002s  


In [2]:
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/train.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/dev.txt'
!mkdir data && mv train.txt data && mv test.txt data && mv dev.txt data

train.txt           100%[===================>]   2.20M  --.-KB/s    in 0.1s    
test.txt            100%[===================>] 108.10K  --.-KB/s    in 0.02s   
dev.txt             100%[===================>] 160.83K  --.-KB/s    in 0.03s   


In [3]:
!cat data/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > train.txt.tmp
!cat data/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > test.txt.tmp
!cat data/dev.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > dev.txt.tmp

In [4]:
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install seqeval
!pip -q install conllu

     |████████████████████████████████| 1.9MB 5.5MB/s 
     |████████████████████████████████| 890kB 19.3MB/s 
     |████████████████████████████████| 3.2MB 34.4MB/s 
     |████████████████████████████████| 1.2MB 4.2MB/s 
     |████████████████████████████████| 51kB 3.1MB/s 


In [5]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2021-03-05 07:55:54--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2021-03-05 07:55:54 (58.5 MB/s) - ‘preprocess.py’ saved [991/991]



In [6]:
!python3 preprocess.py dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > dev.txt
!python3 preprocess.py train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > train.txt
!python3 preprocess.py test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > test.txt

Downloading: 100% 434/434 [00:00<00:00, 394kB/s]
Downloading: 100% 1.22M/1.22M [00:00<00:00, 14.8MB/s]


In [7]:
!mkdir processed_data
!mv train.txt processed_data/ && mv test.txt processed_data/ && mv dev.txt processed_data/

In [8]:
!cat processed_data/train.txt processed_data/test.txt processed_data/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [ ]:
!python3 run_ner.py \
--data_dir_twitter ./processed_data/ \
--data_dir_peyma ./processed_data/ \
--labels_twitter labels.txt \
--labels_peyma labels.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length_twitter  256 \
--max_seq_length_peyma  256 \
--per_device_train_batch_size 8 \
--save_steps 1000 \
--save_total_limit 1 \
--eval_steps 100 \
--logging_steps 100 \
--logging_first_step True \
--evaluation_strategy steps \
--seed 1 \
--num_train_epochs 5 \
--do_train \
--do_eval \
--do_predict

2021-03-05 07:56:25.527234: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/05/2021 07:56:26 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
03/05/2021 07:56:26 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=eval/, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=EvaluationStrategy.STEPS, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Mar05_07-56-26_f9db250eae4c, logging_first_step=True, logging_steps=100, save_steps=1000, save_total_limit=1, no_cu